In [1]:
import numpy as np
import pandas as pd
import datetime

from datasets import load_data, create_test_data, split_train_data
from features import build_training_point, create_training_points
from modelisation import simulate, LSTNet



In [2]:
#Récupération des données
files = ["SPX.txt","VIX.txt","USGG2YR.txt"]
df = load_data(files)
df.head()

,Date,Prix_SPX,Prix_VIX,Prix_USGG2YR
0,1995-01-06,460.68,13.13,7.621
1,1995-01-13,465.97,11.10,7.341
2,1995-01-20,464.78,12.15,7.479
3,1995-01-27,470.39,11.25,7.244
4,1995-02-03,478.64,10.98,7.138


In [3]:
#Le modèle sera evalué sur la prochaine seulement
horizon_weeks = 1
epochs = 3000
#Le modele sera entrainé sur 30 semaines de données
history_weeks = 30
#Le modèle sera entrainé sur les données avant 2013
split = 2013

In [4]:
#Conversion en jours
horizon_days = horizon_weeks *7
history_days = history_weeks * 7

#Séparation des données de train
train_data = split_train_data(df, split_year=split)

#Créé les points pour le test
X_test, Y_test = create_test_data(df, history_days=history_days, horizon_days=horizon_days, date = datetime.datetime(split,1,1))

#Créé les points pour l'entrainement
X_train, Y_train = create_training_points(train_data, history_days=history_days, horizon_days=horizon_days)


#Configuration du model
model = LSTNet(history_weeks, horizon_weeks, hidC = 30,Ck = 4,hidR = 40,hidS = 50,skip = 6,loss = "mse",pt =int((history_weeks-3)/15))

#Entrainement
history= []
period = 30
for i in range(int(epochs/period)):
    history.append(model.fit(X_train, Y_train, epochs = period, batch_size = X_train.shape[0], verbose = False))
    print("epoch: ", i * period,"/", epochs,"loss: ", history[-1].history["loss"][-1], "mse:", history[-1].history["mean_squared_error"][-1])
model.evaluate(X_test, Y_test)

epoch:  0 / 3000 loss:  278.7892150878906 mse: 275.6979675292969
epoch:  30 / 3000 loss:  152.40602111816406 mse: 150.2242889404297
epoch:  60 / 3000 loss:  94.6294937133789 mse: 93.16078186035156
epoch:  90 / 3000 loss:  82.21898651123047 mse: 81.2787094116211
epoch:  120 / 3000 loss:  75.91191864013672 mse: 75.28423309326172
epoch:  150 / 3000 loss:  58.621829986572266 mse: 58.091697692871094
epoch:  180 / 3000 loss:  47.43673324584961 mse: 46.89222717285156
epoch:  210 / 3000 loss:  41.11812210083008 mse: 40.55311584472656
epoch:  240 / 3000 loss:  37.21280288696289 mse: 36.63976287841797
epoch:  270 / 3000 loss:  35.061370849609375 mse: 34.48540115356445
epoch:  300 / 3000 loss:  34.60295104980469 mse: 34.018882751464844
epoch:  330 / 3000 loss:  30.623598098754883 mse: 30.0285587310791
epoch:  360 / 3000 loss:  29.875497817993164 mse: 29.247045516967773
epoch:  390 / 3000 loss:  28.576322555541992 mse: 27.93189239501953
epoch:  420 / 3000 loss:  26.656740188598633 mse: 25.99667739

[17.268638610839844, 16.41539764404297]

In [5]:
## Définition des 2 stratégies
#Première stratégie, parié en fonction des résultats du modèle en prévoyant une fenètre d'erreur ou il est trop risqué de prédire
def strategy1(y_pred, X_test, eps = 0.5):
    last_VIX = X_test[0,int(history_days/7 - 1),1]
    if last_VIX - y_pred > eps:
        return -1 #parie a la baisse
    if  y_pred - last_VIX > eps:
        return 1 #parie a la hausse
    return 0

#Deuxième stratégie, parie aléatoire
def random_strategy(y_pred, X_test):
    return np.random.randint(-1,2)

In [6]:
datestart = datetime.datetime(split,1,1)
random_gain = simulate(random_strategy, df, horizon_days, history_days, datestart, model)        
print("Avec une stratégie aléatoire, le gain est de ", random_gain)
gain1 = simulate(strategy1, df, horizon_days, history_days, datestart, model)   
print("Avec la stratégie 1, le gain est de ", gain1)

Avec une stratégie aléatoire, le gain est de  0.7600000000000016
Avec la stratégie 1, le gain est de  195.28
